In [9]:
import requests
import json
import math
import time

# 1. 인증키 (Decoding된 키)
service_key = "58789b62af0cd3a650734b7011a4005381f84821ee32b63f81bf9969dcefd54e"
base_url = "https://apis.data.go.kr/1613000/AptListService3/getSigunguAptList3"

target_districts = {
    "송파구": "11710",
    "마포구": "11440",
    "노원구": "11350"
}

final_result = {} # 결과를 담을 딕셔너리

for district_name, code in target_districts.items():
    print(f"\n=== [{district_name}] 수집 시작 ===")
    
    all_items = []
    page = 1
    
    while True:
        params = {
            'serviceKey': service_key,
            'pageNo': page,
            'numOfRows': 100,   # 한 번에 100개씩 (최대치)
            'sigunguCode': code,
            '_type': 'json'
        }

        try:
            response = requests.get(base_url, params=params, timeout=10)
            
            if response.status_code != 200:
                print(f"❌ 요청 실패 (Page {page}): Code {response.status_code}")
                break
                
            data = response.json()
            
            # 데이터 파싱 (구조: response -> body -> items -> item)
            # 데이터가 없을 경우(API 에러 등) 방어 코드
            if 'response' not in data or 'body' not in data['response']:
                 print(f"❌ 데이터 구조 이상: {data}")
                 break

            body = data['response']['body']
            items = body.get('items', [])
            total_count = body.get('totalCount', 0)
            
            # 중요: items가 딕셔너리 하나로 올 때가 있고, 리스트로 올 때가 있음 (공공데이터 종특)
            # 리스트가 아니면 리스트로 감싸줌
            if isinstance(items, dict): 
                items = [items]
            
            # 가져온 데이터 저장
            all_items.extend(items)
            
            print(f" -> {page}페이지 완료 (누적 {len(all_items)} / 전체 {total_count})")
            
            # [탈출 조건] 현재까지 모은 개수가 전체 개수 이상이면 그만!
            if len(all_items) >= total_count:
                print(f"✅ {district_name} 수집 완료! (총 {len(all_items)}개)")
                break
                
            # [탈출 조건 2] 데이터가 더 이상 안 오면 그만!
            if not items:
                print("⚠️ 더 이상 데이터가 없음")
                break

            page += 1
            
        except Exception as e:
            print(f"⚠️ 에러 발생: {e}")
            break
            
    # 결과 저장
    final_result[district_name] = all_items

print("\n🎉 전체 작업 종료")


=== [송파구] 수집 시작 ===
 -> 1페이지 완료 (누적 100 / 전체 173)
 -> 2페이지 완료 (누적 173 / 전체 173)
✅ 송파구 수집 완료! (총 173개)

=== [마포구] 수집 시작 ===
 -> 1페이지 완료 (누적 100 / 전체 139)
 -> 2페이지 완료 (누적 139 / 전체 139)
✅ 마포구 수집 완료! (총 139개)

=== [노원구] 수집 시작 ===
 -> 1페이지 완료 (누적 100 / 전체 225)
 -> 2페이지 완료 (누적 200 / 전체 225)
 -> 3페이지 완료 (누적 225 / 전체 225)
✅ 노원구 수집 완료! (총 225개)

🎉 전체 작업 종료


In [12]:
import pandas as pd
from datetime import datetime

# 데이터를 하나로 합칠 리스트
merged_data = []

# final_result 딕셔너리를 돌면서 데이터 평탄화
# district: "송파구", items: [아파트1, 아파트2...]
for district, items in final_result.items():
    for item in items:
        # 원본 데이터에 '구' 정보를 추가 (나중에 섞여도 구분하기 위함)
        item['자치구'] = district 
        merged_data.append(item)

# 데이터가 있을 때만 저장 수행
if merged_data:
    # 1. 데이터프레임(표)으로 변환
    df = pd.DataFrame(merged_data)
    
    # (옵션) 보기 좋게 '자치구' 컬럼을 맨 앞으로 이동
    cols = ['자치구'] + [c for c in df.columns if c != '자치구']
    df = df[cols]
    
    # 2. 파일명 생성 (오늘 날짜 포함)
    today = datetime.now().strftime("%Y%m%d")
    file_name = f"서울_아파트단지_목록_{today}.csv"
    
    # 3. CSV로 저장
    # encoding='utf-8-sig'가 핵심입니다! (이게 없으면 엑셀에서 한글이 꺠져 보입니다)
    df.to_csv(file_name, index=False, encoding='utf-8-sig')
    
    print(f"✅ 저장 완료! 파일명: {file_name}")
    print(f"총 {len(df)}개의 아파트 단지가 저장되었습니다.")
    
    # 미리보기 (상위 5개)
    print(df.head())

else:
    print("⚠️ 저장할 데이터가 없습니다. (final_result가 비어있음)")

✅ 저장 완료! 파일명: 서울_아파트단지_목록_20251118.csv
총 537개의 아파트 단지가 저장되었습니다.
   자치구   kaptCode       kaptName     bjdCode    as1  as2  as3   as4
0  송파구  A10023070       잠실 센트럴파크  1171010100  서울특별시  송파구  잠실동  None
1  송파구  A10023599          잠실엘타워  1171010100  서울특별시  송파구  잠실동  None
2  송파구  A10027523  월드메르디앙주상복합아파트  1171010100  서울특별시  송파구  잠실동  None
3  송파구  A13822001       잠실레이크팰리스  1171010100  서울특별시  송파구  잠실동  None
4  송파구  A13822002        잠실동트리지움  1171010100  서울특별시  송파구  잠실동  None
